# purpose : New pretrained file for DeepgazeIIE model 

## data & frame 준비 

In [1]:
# data path
base_path = '/Users/krc/Documents/retail/'

In [2]:
# install check 
import torch
import os
import csv
torch.__version__

'1.13.0.dev20220623'

In [3]:
os.getcwd()

'/Users/krc/Documents/retail/retail_gh/deepgaze_sample'

In [4]:
%cd ../../

/Users/krc/Documents/retail


In [5]:
# #mmdetection installation 
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .
# !pip install mmdet

In [6]:
os.listdir()

['1조_리테일앤인사이트_기획안발표.pptx',
 '리테일앤인사이트_RFP.docx',
 '.DS_Store',
 'DeepGazeIIE-1.0.0',
 'retail_gh',
 'work',
 'mmdetection',
 'deep_gaze_jup',
 '1조_리테일앤인사이트_기획안발표_수정1.pptx',
 'data',
 'reference']

In [7]:
# annotation by csv 
f =open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/annotations_train.csv','r')
annotation = csv.reader(f)
# image_name,x1,y1,x2,y2,class,image_width,image_height
ls_ann = list(annotation)
print(ls_ann)
f.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
index = ['image_name','x1','y1','x2','y2','class','image_width','image_height']
print(index)
print(ls_ann[-1])
print(ls_ann[-1][0])


['image_name', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height']
['train_999.jpg', '905', '1589', '1144', '1712', 'object', '2336', '4160']
train_999.jpg


In [21]:
# annotation 생성
import copy
import os.path as osp
import cv2
import yaml

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

CLASSES = ('object')
# cat2label = {k:i for i, k in enumerate(CLASSES)}

# 반드시 아래 Decorator 설정 할것.@DATASETS.register_module() 설정 시 force=True를 입력하지 않으면 Dataset 재등록 불가. 
# @DATASETS.register_module(force=True)
# class SKIKU110K(CustomDataset):
#   CLASSES = ('object')
  
  ##### self.data_root: ./kitti_tiny/ self.ann_file: ./kitti_tiny/train.txt self.img_prefix: ./kitti_tiny/training/image_2
  #### ann_file: ./kitti_tiny/train.txt
  # annotation에 대한 모든 파일명을 가지고 있는 텍스트 파일을 __init__(self, ann_file)로 입력 받고, 이 self.ann_file이 load_annotations()의 인자로 입력
def load_annotations(self, ann_file):
#     print('##### self.data_root:', self.data_root, 'self.ann_file:', self.ann_file, 'self.img_prefix:', self.img_prefix)
#     print('#### ann_file:', ann_file)
#    cat2label = {k:i for i, k in enumerate(self.CLASSES)}
    # 포맷 중립 데이터를 담을 list 객체
    data_infos = []
    
    for label in ann_file:
      filename = label[0]
      height, width = label[6],label[7]
      # 개별 image의 annotation 정보 저장용 Dict 생성. key값 filename 에는 image의 파일명만 들어감(디렉토리는 제외)
      data_info = {'filename': str(filename),
                   'width': int(width), 'height': int(height)}

      # 오브젝트의 클래스명은 bbox_names로 저장. 
      bbox_names = label[5]
      # bbox 좌표를 저장
      bboxes = [int(label[1]),int(label[2]), int(label[3]) -int(label[1]), int(label[4])-int(label[2])]

# 굳이? 
#       # 클래스명이 해당 사항이 없는 대상 Filtering out, 'DontCare'sms ignore로 별도 저장.
#       gt_bboxes = []
#       gt_labels = []
#       gt_bboxes_ignore = []
#       gt_labels_ignore = []

#       for bbox_name, bbox in zip(bbox_names, bboxes):
#         # 만약 bbox_name이 클래스명에 해당 되면, gt_bboxes와 gt_labels에 추가, 그렇지 않으면 gt_bboxes_ignore, gt_labels_ignore에 추가
#         if bbox_name in cat2label:
#           gt_bboxes.append(bbox)
#           # gt_labels에는 class id를 입력
#           gt_labels.append(cat2label[bbox_name])
#         else:
#           gt_bboxes_ignore.append(bbox)
#           gt_labels_ignore.append(-1)
      # 개별 image별 annotation 정보를 가지는 Dict 생성. 해당 Dict의 value값은 모두 np.array임. 
      data_anno = {
          'bboxes': np.array(bboxes, dtype=np.float32).reshape(-1, 4).tolist(),
          'labels':[0],
      }
      # image에 대한 메타 정보를 가지는 data_info Dict에 'ann' key값으로 data_anno를 value로 저장. 
      data_info.update(ann=data_anno)
      # 전체 annotation 파일들에 대한 정보를 가지는 data_infos에 data_info Dict를 추가
      data_infos.append(data_info)

    return data_infos

In [22]:

# new_ann = load_annotations(CustomDataset,ann_file=ls_ann)
# print(new_ann)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
# import json
 
# with open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/train_ann.json', 'w') as f:
#     json.dump(new_ann, f,indent=4)

In [24]:
# # annotation by csv 
# f =open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/annotations_val.csv','r')
# annotation = csv.reader(f)
# # image_name,x1,y1,x2,y2,class,image_width,image_height
# val_ann = list(annotation)
# f.close()

# import json
# val_ann = load_annotations(CustomDataset,ann_file=val_ann)


# with open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/val_ann.json', 'w') as f:
#     json.dump(val_ann, f,indent=4)

In [13]:
os.getcwd()

'/Users/krc/Documents/retail'

## DETR 사전 가중치 다운로드 

In [14]:
os.chdir('/Users/krc/Documents/retail/mmdetection/')
if not os.path.isdir("checkpoint"):
    os.mkdir("checkpoint")
os.chdir('checkpoint')

from requests import get 

def download(url, file_name):
    with open(file_name, "wb") as file:   
        response = get(url)               
        file.write(response.content)


# url = "https://github.com/open-mmlab/mmdetection/tree/master/configs/deformable_detr"
# download(url,"deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth.pth")

os.chdir('/Users/krc/Documents/retail/mmdetection/')

## wand (굳이..) 

In [25]:
#wandb setting first 
import wandb

In [26]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [31]:
#!pip install mmcv-full
#!pip install terminaltables
#!pip install imagecorruptions
#!pip install -q pycocotools

import mmcv
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector
from mmdet.apis import set_random_seed
import os.path as osp

def set_config():
  # 사용할 모델 초기 py 설정
  config = '/Users/krc/Documents/retail/mmdetection/configs/deformable_detr/deformable_detr_r50_16x2_50e_coco.py'

  # config 불러오기
  cfg = mmcv.Config.fromfile(config)

  # config 에 base 기록
  cfg.base = '/Users/krc/Documents/retail/mmdetection/configs/detr/detr_r50_8x2_150e_coco.py'

  # 데이터셋 지정
  cfg.dataset_type = 'CustomDataset'

  # data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
  data_root = '/Users/krc/Documents/retail/data/SKU110K_fixed/'

  # class 설정
  cfg.classes = ('object',)

  # train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

  cfg.data.samples_per_gpu=1
  cfg.data.workers_per_gpu=0

  cfg.data.train.data_root = data_root
  cfg.data.train.ann_file = '/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/train_ann.json' 
  cfg.data.train.img_prefix = 'train/' 
  cfg.data.train.classes = cfg.classes
  cfg.data.train.type = 'CustomDataset'

  cfg.data.val.data_root = data_root 
  cfg.data.val.ann_file = '/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/val_ann.json'
  cfg.data.val.img_prefix = 'val/'
  cfg.data.val.classes = cfg.classes
  cfg.data.val.type = 'CustomDataset'

  cfg.data.test.data_root = data_root 
  cfg.data.test.ann_file = 'test.json'
  cfg.data.test.img_prefix = 'test/'
  cfg.data.test.classes = cfg.classes
  cfg.data.test.type = 'CustomDataset'

  # 클래스 수 지정
  cfg.model.bbox_head.num_classes = 1
#  cfg.model.roi_head.mask_head.num_classes = 1


  # 사전 훈련 모델 지정
  cfg.checkpoint_config = dict(interval=10,out_dir='work/ch_config/')
  cfg.load_from = '/Users/krc/Documents/retail/mmdetection/checkpoint/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth'


  # 가중치 저장 위치
  cfg.work_dir =  './work/weights/'

  # 평가 지표로 설정
  cfg.evaluation.metric = ['bbox']

  # epoch 설정
  cfg.runner = dict(type='EpochBasedRunner', max_epochs=40) 

  # batch size 설정
  cfg.auto_scale_lr = dict(enable=False, base_batch_size=32)

  # gpu 설정 및 seed 설정
  cfg.seed = 42
  cfg.gpu_ids = range(1)
  cfg.device= 'cpu' # for local

  #wandb setting
#   cfg.log_config.hooks = [
#         dict(type='TextLoggerHook'),
#         dict(type='MMDetWandbHook',
#              init_kwargs={
#                           'project': 'SKU_DETR',
#                           'config': {"optimizer" : 'AdamW, default',
#                           "batch_size" :32,
#                           "epochs" :40,
#                           "lr_config" : 'CosineAnnealning' }
#                          },
#              interval=4,
#              log_checkpoint=True,
#              log_checkpoint_metadata=True,
#              num_eval_images=100,
#              bbox_score_thr=0.1)]

  #learning rate
  cfg.lr_config = dict(
   policy='CosineAnnealing',
   by_epoch=False,
   min_lr=0)
    
    
  meta = dict()
  meta['exp_name'] = osp.basename(config)
  print(meta)


  return cfg



In [32]:
cfg = set_config()

{'exp_name': 'deformable_detr_r50_16x2_50e_coco.py'}


In [33]:
print(cfg.pretty_text)

dataset_type = 'CustomDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[[{
            'type':
            'Resize',
            'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333),
                          (608, 1333), (640, 1333), (672, 1333), (704, 1333),
                          (736, 1333), (768, 1333), (800, 1333)],
            'multiscale_mode':
            'value',
            'keep_ratio':
            True
        }],
                  [{
                      'type': 'Resize',
                      'img_scale': [(400, 4200), (500, 4200), (600, 4200)],
                      'multiscale_mode': 'value',
                      'keep_ratio': True
                  }, {
              

## train

In [34]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# dataset 생성 및 model 설정

datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES
# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)



2022-10-11 18:12:44,067 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-10-11 18:12:44,370 - mmdet - INFO - load checkpoint from local path: /Users/krc/Documents/retail/mmdetection/checkpoint/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth
2022-10-11 18:12:45,027 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.cls_branches.0.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
size mismatch for bbox_head.cls_branches.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
size mismatch for bbox_head.cls_branches.1.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
size mismatch for bbox_head.cls_branches.1.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape

RuntimeError: expected scalar type Float but found Double

Thread SenderThread:
Traceback (most recent call last):
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 50, in run
    self._run()
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 101, in _run
    self._process(record)
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/internal.py", line 308, in _process
    self._sm.send(record)
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/sender.py", line 305, in send
    send_handler(record)
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/sender.py", line 319, in send_request
    send_handler(record)
  File "/Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/wandb/sdk/internal/sender.py", line 486, in send_request_defer
    transition_state()
  File "/Users/krc/miniforge3/envs/pytorch/